In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
from math import radians, cos, sin, asin, sqrt
%matplotlib inline

from scipy.spatial import cKDTree
from tqdm import tqdm


In [2]:
data = pd.read_csv (r'osm_data.csv')
df = pd.DataFrame(data, columns = ['id', 'latitude', 'longitude', 'description'])

In [3]:
vnf_df = pd.read_csv(r'vnf_2017_label_DBSCANFinalWith0.0002_onlyCenterMostPoint.csv')
# vnf_df = pd.DataFrame(vnf_data, columns=['id_Key', 'Date_Mscan','Lat_GMTCO','Lon_GMTCO','Temp_BB', 'RHI', 'RH', 'Area_BB', 'Cloud_Mask', 'Sample_M10'])
vnf_df.drop(vnf_df[vnf_df['Temp_BB'] == 999999].index, inplace = True)
vnf_lats = vnf_df['Lat_GMTCO']
vnf_long = vnf_df['Lon_GMTCO']                   

In [4]:
vnf_df.head()

,Unnamed: 0,id_Key,Date_Mscan,Lat_GMTCO,Lon_GMTCO,Temp_BB,RHI,RH,Area_BB,Cloud_Mask,...,Area_BB_Mean_Label,Area_BB_StdLabel,Area_BB_Label_Day,RHI_Max_Label,RHI_Min_Label,RHI_Mean_Label,RHI_StdLabel,RHI_Label_Day,Lat_GMTCO_Label,Lon_GMTCO_Label
0,0,x0836225E_y684091N_l0597_s2762_v21,2017/11/10 23:37:00.763,68.409103,83.622475,1783.0,40.469900,40.249200,70.286400,1.0,...,12.933069,26.842369,70.28640,111.1510,0.257765,6.203263,11.458823,46.87520,68.409103,83.622475
1,1,x0835226E_y677973N_l1694_s2077_v21,2017/02/16 22:03:17.110,67.797295,83.522552,2172.0,0.415647,0.346884,0.274937,3.0,...,6.616613,14.273149,68.47960,55.3995,0.259088,1.807915,3.341784,55.39950,67.797295,83.522552
2,2,x0833993E_y678584N_l0804_s0125_v21,2017/01/03 19:07:07.003,67.858391,83.399261,1981.0,1.767480,3.390210,3.884210,0.0,...,4.013785,4.876577,9.20320,25.9740,0.268409,2.061008,2.083913,3.12737,67.858391,83.399261
3,3,x0818293E_y696207N_l0281_s1953_v21,2017/11/20 22:09:36.446,69.620728,81.829292,1931.0,2.766030,1.940870,2.460260,1.0,...,5.304818,5.417266,7.16263,14.9619,0.261412,2.244051,1.584091,3.79056,69.620728,81.829292
4,4,x0799487E_y685164N_l0917_s0513_v21,2017/02/28 19:57:12.991,68.516411,79.948692,1730.0,3.882040,3.433330,6.756840,0.0,...,6.489901,13.713032,16.31860,78.6269,0.246062,3.583848,5.918761,13.00450,68.516411,79.948692


In [5]:
def haversine_np( lat1, lon1,  lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
#     dlon = np.expand_dims(lon2,axis = 0) - np.expand_dims(lon1,axis = 1)
#     dlat = np.expand_dims(lat2,axis = 0) - np.expand_dims(lat1,axis = 1)
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return min(km)

In [6]:
osm_points = list(zip(df['latitude'], df['longitude']))
osm_points_np = np.array(osm_points)
tree = cKDTree(osm_points_np)
def close_to_osm_oil(row, radius = 6):
#     print(row)
    lat1 = row['Lat_GMTCO']
    lon1 = row['Lon_GMTCO']
    # Find k = 10 nearest neighbours in euclidean(!!!) distance
    dd, ii = tree.query(np.array([lat1,lon1]), k = 500)
    # Check if the spherical closest neighbour among the k = 10 euclidean nearest neighbours 
    # is nearer than 'radius' kilometres
    if haversine_np(lat1, lon1,osm_points_np[ii][:,0],osm_points_np[ii][:,1]) <= radius :
        return True
    else :
        return False
def not_close_to_osm_oil(row, radius = 50):
#     print(row)
    lat1 = row['Lat_GMTCO']
    lon1 = row['Lon_GMTCO']
    # Find k = 10 nearest neighbours in euclidean(!!!) distance
    dd, ii = tree.query(np.array([lat1,lon1]), k = 500)
    # Check if the spherical closest neighbour among the k = 10 euclidean nearest neighbours 
    # is nearer than 'radius' kilometres
    if haversine_np(lat1, lon1,osm_points_np[ii][:,0],osm_points_np[ii][:,1]) > radius :
        return True
    else :
        return False


In [7]:
from tqdm import tqdm
tqdm.pandas(desc="my bar!")
is_close = vnf_df.progress_apply(close_to_osm_oil, axis = 1)


my bar!: 100%|██████████| 449943/449943 [02:06<00:00, 3563.45it/s]


In [8]:
is_not_close = vnf_df.progress_apply(not_close_to_osm_oil, axis = 1)

my bar!: 100%|██████████| 449943/449943 [02:10<00:00, 3435.00it/s]


In [9]:
vnf_df["osm_flare"] = is_close
result_df = vnf_df[is_close | is_not_close]
result_df.to_csv("flare_labelled_vnf")

In [10]:
result_df["osm_flare"].value_counts()

False    427256
True       3401
Name: osm_flare, dtype: int64

In [25]:
result_df[(result_df["Temp_BB"]< 1400) & (result_df["osm_flare"])] 

,Unnamed: 0,id_Key,Date_Mscan,Lat_GMTCO,Lon_GMTCO,Temp_BB,RHI,RH,Area_BB,Cloud_Mask,...,Area_BB_StdLabel,Area_BB_Label_Day,RHI_Max_Label,RHI_Min_Label,RHI_Mean_Label,RHI_StdLabel,RHI_Label_Day,Lat_GMTCO_Label,Lon_GMTCO_Label,osm_flare
795,795,x0901659E_y430715N_l0325_s0880_v21,2017/04/19 19:26:05.044,43.071541,90.165947,1179.0,2.645250,2.269030,20.69900,0.0,...,11.498362,20.69900,8.690870,0.338408,2.069114,1.610287,2.645250,43.071541,90.165947,True
895,895,x0432363E_y358412N_l1085_s3170_v21,2017/01/10 00:06:36.745,35.841202,43.236282,1162.0,3.539430,8.729960,84.39380,0.0,...,72.265062,293.79700,74.301300,0.529858,8.557616,9.642415,21.211200,35.841202,43.236282,True
903,903,x1154445E_y207837S_l0141_s2879_v21,2017/05/15 18:14:49.324,-20.783674,115.444527,1316.0,4.435700,5.437210,31.99320,0.0,...,59.347773,44.34910,118.748000,0.255013,6.670684,10.692474,6.746400,-20.783674,115.444527,True
921,921,x0400666W_y225484S_l0510_s1660_v21,2017/03/04 03:53:07.029,-22.548443,-40.066589,1217.0,1.606670,0.930166,7.48079,0.0,...,19.619375,7.48079,105.920000,0.320544,4.869243,9.558065,1.606670,-22.548443,-40.066589,True
927,927,x0978092W_y222691N_l0361_s2711_v21,2017/05/28 08:50:36.172,22.269133,-97.809196,1204.0,1.959550,1.782270,14.93500,0.0,...,11.371186,16.96310,45.023200,0.354614,4.640643,4.729933,3.000810,22.269133,-97.809196,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446942,446942,x0468313E_y245148N_l0089_s2671_v30,2017/12/30 23:12:31.115,24.514805,46.831348,1226.0,0.971261,0.830299,6.48564,0.0,...,NaN,6.48564,0.971261,0.971261,0.971261,NaN,0.971261,24.514805,46.831348,True
446998,446998,x1114940E_y603775N_l0195_s1908_v30,2017/12/31 19:20:26.109,60.377533,111.493988,1197.0,2.327170,1.555730,13.38250,0.0,...,4.077170,13.38250,2.327170,0.516568,1.221911,0.969288,2.327170,60.377533,111.493988,True
447138,447138,x0588007E_y438485N_l0347_s0775_v30,2017/12/30 21:26:17.772,43.848507,58.800682,1262.0,3.395810,3.586160,24.93160,0.0,...,6.506231,24.93160,3.395810,1.581280,2.235157,1.007843,3.395810,43.848507,58.800682,True
447881,447881,x0793623E_y615000N_l0434_s1728_v30,2017/12/30 21:20:46.827,61.500000,79.362297,1060.0,1.677480,0.990489,13.86020,0.0,...,NaN,13.86020,1.677480,1.677480,1.677480,NaN,1.677480,61.500000,79.362297,True
